# Ukraine’s military intelligence list of ru FSB officers

Ukraine’s military intelligence on 28 March 2022 published the names and contact details of 620 people it alleged were **officers of Russia’s Federal Security Service (FSB)** involved in “criminal activities” in Europe.

The list is available on the official webpage of the Ukrainian Military Intelligence Service:
* https://gur.gov.ua/content/sotrudnyky-fsb-rossyy-uchastvuiushchye-v-prestupnoi-deiatelnosty-stranyahressora-na-terrytoryy-evropy.html

Next to the names there is data that the Ukrainian special service claims to contain **date of birth, passport, address registration, debts, SIM cards, modems, cars, special remarks and more**. All 620 agents are registered at the same address – Bolshaya Lubyanka Street in Moscow, the address of FSB, the former KGB.

In [1]:
from bs4 import BeautifulSoup as bs
import requests
import urllib.request
import re
import pandas as pd

from transliterate import translit
from googletrans import Translator

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36'
}

url = 'https://gur.gov.ua/content/sotrudnyky-fsb-rossyy-uchastvuiushchye-v-prestupnoi-deiatelnosty-stranyahressora-na-terrytoryy-evropy.html'

response = requests.get(url, headers=headers)
soup = bs(response.content, 'html.parser')

In [3]:
p = soup.find_all('p')

In [5]:
data = []

for person in p:
    #print(person.text)
    data.append(person.text)
    #print("*************************************************************************************")
    #print("*************************************************************************************")
    #print("*************************************************************************************")
    #print("*************************************************************************************")

In [7]:
# example
data[:5]

['Ф.И.О., Дата рождения, Место рождения, Паспорт, Дата выдачи, Орган выдачи, Код органа, Адрес регистрации, Орган, Адрес ранее, Особые отметки, СИГНАТУРА, Долги, Авиабилеты, Походы за кредитами, Сим-карты, модемы, Автотранспорт, Системный номер, Признак "продано ранее"',
 '1. АБАДЖИ МИХАИЛ ВАЛЕРЬЕВИЧ, 22.03.1977, 3802688956, 01.10.2002, МОСКВА, Б.ЛУБЯНКА д.2 ВЧ, Федеральная служба безопасности (ФСБ), РФ, +79773401033, E54FD81C33BF8586B151B2ABD2345EBE, ТВ(1), 2395;',
 '2. АБГАРЯН АРМАН ХАЧАТУРОВИЧ, 12.06.1981, 101000 г. Москва, ул. Б. Лубянка, д. 2, Федеральная служба безопасности (ФСБ) РФ, A3B52E412CB1B1E0FC3D3B859B191073, 2525;',
 '3. АБРАМОВ РОМАН ВИКТОРОВИЧ, 05.12.1986, 9006759816, 107031, г.Москва, ул.Большая Лубянка, дом 1/3, Федеральная служба безопасности (ФСБ), РФ, 2077754864E9763D0FD62006489D09FC, АА(8), АВ(3), 44, +;',
 '4. АБРАМОВИЧ СЕРГЕЙ ВИТАЛИЕВИЧ, 15.08.1971, 4499013179, 23.05.2000, УВД МАГАДАН, МОСКВА, ЛУБЯНКА Б. д.1/3 вед., Федеральная служба безопасности (ФСБ), РФ, 86

In [8]:
df = pd.DataFrame(data)

df.columns = ["person info"]

df.drop(df.index[0], inplace=True)

For translating from Ukrainian to English, I have used googletrans library.

In [9]:
def translate_(text):
    translator = Translator()
    try:
        translation = translator.translate(text, src='uk', dest='en')
        return translation.text
    except Exception as e:
        print(f"Translation failed for '{text}': {e}")
        return text

def to_english(df, column_name):
    df[column_name] = df[column_name].apply(translate_)
    return df

In [10]:
df = to_english(df, "person info")

In [11]:
for i in df['person info'][:5]:
    print(i)
    print("*******************")

1. Abadhi Mikhail Valerievich, 22.03.1977, 3802688956, 01.10.2002, Moscow, B. Lubyanka D.2 HF, Federal Security Service (FSB), Russian Federation, +797734010333 1), 2395;
*******************
2. Abgarian Arman Khachaturovich, 12.06.1981, 101000 Moscow, ul.B. Lubyanka, 2, Federal Security Service (FSB) of the Russian Federation, A3B52E412CB1B1B1B1E0FC3D3B859B191073, 2525;
*******************
3. АБРАМОВ РОМАН ВИКТОРОВИЧ, 05.12.1986, 9006759816, 107031, г.Москва, ул.Большая Лубянка, дом 1/3, Федеральная служба безопасности (ФСБ), РФ, 2077754864E9763D0FD62006489D09FC, АА(8), АВ(3), 44, +;
*******************
4. Abramovich Sergey Vitaliyevich, 15.08.1971, 4499013179, 23.05.2000, Magadan, Moscow, Lubyanka BD 1/3 Ved. , TV (1),2069;
*******************
5. Avezov Andrey Makhudovich, 07.12.1976, 4901171018, 30.08.2001, Moscow, Lubyanka BD 1, Federal Security Service (FSB), RF, 5D8FB47096827, 238282709
*******************


#### Below, there are some further data transformations.

In [12]:
# removing any character that is a digit or a point (.) from the first 6 characters
def delete_dg(text):
    cleaned_text = ''.join(char for char in text[:6] if not char.isdigit() and char != '.')
    cleaned_text += text[6:]
    return cleaned_text

df['person info'] = df['person info'].apply(delete_dg)

df = df[df['person info'].str.len() >= 70]

In [13]:
for i in df['person info'][:5]:
    print(i)
    print("***************************************************")

 Abadhi Mikhail Valerievich, 22.03.1977, 3802688956, 01.10.2002, Moscow, B. Lubyanka D.2 HF, Federal Security Service (FSB), Russian Federation, +797734010333 1), 2395;
***************************************************
 Abgarian Arman Khachaturovich, 12.06.1981, 101000 Moscow, ul.B. Lubyanka, 2, Federal Security Service (FSB) of the Russian Federation, A3B52E412CB1B1B1B1E0FC3D3B859B191073, 2525;
***************************************************
 АБРАМОВ РОМАН ВИКТОРОВИЧ, 05.12.1986, 9006759816, 107031, г.Москва, ул.Большая Лубянка, дом 1/3, Федеральная служба безопасности (ФСБ), РФ, 2077754864E9763D0FD62006489D09FC, АА(8), АВ(3), 44, +;
***************************************************
 Abramovich Sergey Vitaliyevich, 15.08.1971, 4499013179, 23.05.2000, Magadan, Moscow, Lubyanka BD 1/3 Ved. , TV (1),2069;
***************************************************
 Avezov Andrey Makhudovich, 07.12.1976, 4901171018, 30.08.2001, Moscow, Lubyanka BD 1, Federal Security Service (FSB), RF, 5D

Creating a new column with the name of the persons

In [14]:
names_list = df["person info"].apply(lambda x: x.split(",", 1)[0]) # split by comma

list_ = names_list.tolist()

df.loc[:, "name"] = list_

df = df[['name', 'person info']]

def remove_first_element(s):
    elements = s.split(',') # split the string by comma (",")
    result = ','.join(elements[1:]) # remove the first element and join the rest back into a string
    return result

df['person info'] = df['person info'].apply(remove_first_element)

Keeping only valid names.

In [15]:
def keep_valid_names(text): 
    pattern = r'^[A-Za-z\s-]+$' # pattern to match valid names (letters, spaces and hyphens)
    return text if re.match(pattern, text) else ''

df['name'] = df['name'].apply(keep_valid_names)

df = df[df['name'].str.len() >= 3]

In [23]:
df.head(10)

,name,person info
1,Abadhi Mikhail Valerievich,"22.03.1977, 3802688956, 01.10.2002, Moscow, B..."
2,Abgarian Arman Khachaturovich,"12.06.1981, 101000 Moscow, ul.B. Lubyanka, 2,..."
4,Abramovich Sergey Vitaliyevich,"15.08.1971, 4499013179, 23.05.2000, Magadan, ..."
5,Avezov Andrey Makhudovich,"07.12.1976, 4901171018, 30.08.2001, Moscow, L..."
6,Agooltsev Mikhail Sergeyevich,"05.07.1986, no Dannykh, 107031, G. Moscow, Vl..."
7,Azimov Ramydin Emirzhanovich,"12.08.1971, 8202913216, 25.10.2002, G. Makhac..."
8,Idarov Tatiana Yuryevna,"05.11.1971, Altai Krai, Aleysky District, S. ..."
9,Maxim Alexandrovich Akinin,"05.02.1981, 6103908757, 07.07.2003, Zheleznor..."
10,Aksenov Oleg Alekseevich,"26.03.1967, Belgorod, 1404397559, 03.04.2004,..."
11,Aksenov Fedor Olegovich,"4510826767, 14.12.2010, Officing of UFMS Russ..."


In [22]:
for i in df.iloc[0]:
    print(i)
    print('********************')

 Abadhi Mikhail Valerievich
********************
 22.03.1977, 3802688956, 01.10.2002, Moscow, B. Lubyanka D.2 HF, Federal Security Service (FSB), Russian Federation, +797734010333 1), 2395;
********************
